<a href="https://colab.research.google.com/github/ykitaguchi77/AutoGen/blob/main/Autogen%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB_%E3%82%B0%E3%83%AB%E3%83%BC%E3%83%97%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%EF%BC%88%E3%83%A1%E3%83%8B%E3%83%A5%E3%83%BC%E8%80%83%E6%A1%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autogenについて
Microsoft Researchから登場したマルチエージェントによるLLMアプリケーションを実現するフレームワーク。役割を設定したエージェントを定義することで、エージェントがお互いに相談しながらタスクを実行できる。インターネットアクセスやコードの実行、ファンクションなども活用しながら高度な操作ができる。

- 公式サイト  
  https://microsoft.github.io/autogen/  
- Github  
  https://github.com/microsoft/autogen  
- ブログ  
  AutoGen: Enabling next-generation large language model applications  
  https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/  
- arxivペーパー  
  AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation  
  https://arxiv.org/abs/2308.08155  

## 注意
- 実行には、OpenAIのAPIキーが必要。
- 複雑な処理や高度な連携にはGPT-4のアクセスが必要。
- コードの修正などが繰り返されるケースではAPIアクセスが多発しコストが嵩むので注意。
- タスクによっては、エージェント間でコードやソースなどを共有する必要があり、大量の入力が発生するため、16Kや32Kに対応したモデルでないと動作しない（Rate Limitエラーにひっかかる）。
- Web情報の取得やデータ分析などは16K、32Kモデル必須。


### STEP 1：準備
pipでインストール。  
ランタイム環境にインストールされてない場合は以下実行。

In [3]:
%%capture --no-stderr
%pip install pyautogen~=0.1.0

### STEP2：設定
設定はOAI_CONFIGファイルから読み込むこともできるが、今回は直接設定する。  
"api_key"にOpenAIのAPIキーを登録しておく。  
Azureを利用する場合は本家のサンプルを参照のこと。  

In [2]:
# APIキー設定スクリプト
# 目的：Google Driveからapi.txtファイルを読み込み、必要なAPIキーを環境変数として設定する
# 使用するAPI：OpenAI API, SerpAPI, Google Custom Search Engine API

# APIの設定
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
#openai_api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]
os.environ["GOOGLE_CSE_ID"] = key[9]
os.environ["GOOGLE_API_KEY"] = key[11]

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import autogen

config_list = [
    {
        "model": "gpt-4o",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
#        "request_timeout": 120,
#        "max_retry_period": 90,
#        "retry_wait_time": 5,
    },
    {
        "model": "gpt-4o-mini",
        "api_key": key[3],
        "api_type": "open_ai",
        "api_base": "https://api.openai.com/v1",
        "api_version": None,
    },
]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### エージェントの定義
今回の例では、以下の４つのエージェントを定義し、1週間分の夕食の献立を相談しながら決めてもらう。  
- floor_manager  
ユーザーとの対話を担当  
- chef  
メニューを考案する  
- doctor  
医学的立場からメニューを修正する  
- kitchen_manager  
メニューに必要な食材などを調達する
  
「system_message=」でエージェントの役割を定義する。  
  
今回はコードの生成と実行はしないが、必要な場合は  
「code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},」  
を利用する。  
  
「groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)」  
で、参加させるエージェントと実行ラウンド数を決める。  
  
人間の介入を受け付ける場合は、以下を変更
「human_input_mode="TERMINATE"」→「human_input_mode="ALWAYS"」

In [ ]:
llm_config = {"config_list": config_list}
floor_manager = autogen.UserProxyAgent(
   name="floor_manager",
   system_message="ChefやDoctor、Kitchen_Managerと相談しながら調理や食材に関する課題を解決してください。満足が得られなければ再度フィードバックをかけてください",
   code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
   human_input_mode="TERMINATE"
)
chef = autogen.AssistantAgent(
    name="chef",
    system_message="世界中の料理を知り尽くした料理人です。健康面はdoctorが検討するので、考慮する必要はありません。味についてのみ検討したレシピを考案し、最高の料理を提供します。考案したメニューをdoctorと相談して、健康面を考慮しながら修正してください。メニューが決定したらfloor_managerに食材リストの提出を依頼してください。",
    llm_config=llm_config,
)
doctor = autogen.AssistantAgent(
    name="doctor",
    system_message="chefが提案したメニューを医学的な立場で検証し、floor_managerに修正を依頼してください。",
    llm_config=llm_config,
)
kitchen_manager = autogen.AssistantAgent(
    name="kitchen_manager",
    system_message="料理にかかる費用や必要な食材や調味料を管理します。chefが考案しメニューから必要な食材を検討し、floor_managerに知らせてください。",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

### チャット開始
議論してもらうテーマを与えると議論が開始される。  
  
実行時に「chef(to chat_manager):」のように誰宛ての会話なのかが表示される点に注目。

In [ ]:
floor_manager.initiate_chat(
    manager, message=
    "私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、明日の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。"
)
# type exit to terminate the chat

floor_manager (to chat_manager):

私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、明日の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
chef (to chat_manager):

かしこまりました。高血圧を考慮せず、まずは味を追求した明日の夕食メニューを考案します。その後、doctorと相談して健康面を考慮した修正を行います。

### 夕食メニュー（味を優先）

#### メインディッシュ: クリーミーなカルボナーラ
- 生クリームとパルメザンチーズで仕上げた濃厚カルボナーラスパゲッティ
- サクッとしたパンチェッタのトッピング
- フレッシュパセリ

#### サイドディッシュ: ガーリックブレッド
- バターたっぷりの自家製ガーリックブレッド

#### サラダ: シーザーサラダ
- ロメインレタス
- クルトン
- パルメザンチーズ
- シーザードレッシング

### 次のステップ

1. 医師と相談し、健康面からの修正を受けた新しいメニューを提案します。
2. 修正メニューに基づいた食材リストを生成し、floor_managerに調達を依頼します。

次は、doctorに健康面の修正をお願いしましょう。

--------------------------------------------------------------------------------
doctor (to chat_manager):

高血圧に配慮し、美味しさを保ちながら健康的なメニューに修正する提案をします。脂肪や塩分を控えめにし、野菜の摂取量を増やす方針で改善していきましょう。

### 医学的観点からの修正提案

#### メインディッシュ: クリーミーなカルボナーラ（修正後）
- 生クリームを使用せず、代わりに低脂肪ミルクと無塩のチキンストックを使用
- パルメザンチーズの量を控えめに
- パンチェッタの代わりに薄切りのターキーや鶏肉の使用を推奨
- 全粒粉スパゲッティを選択

#**公式のサンプル**

###シンプルな会話

In [ ]:
import os
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [ ]:
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

Sure, here is a light-hearted one for you:

Why don't scientists trust atoms?

Because they make up everything!


###Agentを増やす

In [ ]:
cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4o", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [ ]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.",
                           max_turns=10,
                           max_consecutive_auto_reply=1,)

### Good Byeが含まれていたら会話を終了

In [ ]:
joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower(),
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Why don't scientists trust atoms? Because they make up everything!

GOOD BYE!

--------------------------------------------------------------------------------


###数当てゲーム実装

In [ ]:
import os
from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

###ヒトも会話に参加する

In [ ]:
import os
from autogen import ConversableAgent

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = agent_with_number.initiate_chat(
    human_proxy,  # this is the same agent with the number as before
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to human_proxy):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 43
human_proxy (to agent_with_number):

43

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 52
human_proxy (to agent_with_number):

52

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------
Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit

###中止するかどうかをその都度聞く

In [ ]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

#**Autogen Sample Scripts**

Multiagents

https://github.com/Poly186-AI-DAO/AutoGen-Example-Scripts/tree/master



### Multiagent developent of the python code

In [5]:

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager


gpt4o_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list":  [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}],
    "request_timeout": 120,
}
user_proxy = UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
Speak Japanese
''',
)
scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4o_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code. Speak Japanese"""
)
planner = AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
Plans should be in Japanese.
''',
    llm_config=gpt4o_config,
)
executor = UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)
critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4o_config,
)
groupchat = GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50)
manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4o_config)
user_proxy.initiate_chat(
    manager,
    message="""
新規の眼形成外科クリニックを繁盛させる施策を立案
""",
)

Admin (to chat_manager):


新規の眼形成外科クリニックを繁盛させる施策を立案


--------------------------------------------------------------------------------
Planner (to chat_manager):

1. 市場調査とターゲット設定 (科学者)
   - 競合する眼形成外科クリニックを含む市場調査を実施し、ターゲット顧客層を明確にする。
   - 地域の人口統計や競合状況を分析し、成功の可能性を評価する。

2. ウェブサイトとオンライン予約システムの構築 (エンジニア)
   - ユーザーフレンドリーなウェブサイトを設計し、検索エンジン最適化(SEO)を行う。
   - 患者が簡単に予約を取れるオンライン予約システムを構築する。

3. 医療スタッフの採用とトレーニング (科学者)
   - 経験豊富な眼形成外科医と医療スタッフを採用し、最新の技術とサービスについてトレーニングを行う。
   - 患者とのコミュニケーションスキルを向上させるための研修を実施する。

4. マーケティング戦略の策定と実施 (科学者)
   - 地域のターゲット市場に合わせた広告キャンペーンを展開し、ソーシャルメディアを活用したプロモーションを行う。
   - 地元のコミュニティイベントに参加し、地域住民との関係を構築する。

5. 患者のフィードバック収集とサービス改善 (科学者)
   - 患者からのフィードバックを定期的に収集し、クリニックのサービス改善に活用する。
   - 継続的に患者満足度を向上させるための施策を検討・実施する。

6. 技術の最適化と効率化 (エンジニア)
   - クリニックの運営に必要な技術を最適化し、電子カルテやデジタルツールの導入を検討する。
   - 患者データの管理と分析を効率化するシステムを開発する。

この計画に対するフィードバックをお願いします。

--------------------------------------------------------------------------------
Provide feedback to chat_manager. Press enter to skip a

###マルチエージェントの成果物

In [ ]:
### Arxivのリスト化

!pip install arxiv scikit-learn

import arxiv
import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

# 現在の日付と1週間前の日付を取得
now = datetime.datetime.now()
last_week = now - datetime.timedelta(weeks=1)

# 日付範囲の文字列を作成
date_query = f"submittedDate:[{last_week.strftime('%Y%m%d')} TO {now.strftime('%Y%m%d')}]"

# ArXiv APIを使用してLLM関連の論文を検索
search_query = f'("large language model" OR LLM) AND {date_query}'
search = arxiv.Search(
    query=search_query,
    max_results=20,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending,
)

# 論文をリスト化
papers = []
abstracts = []
try:
    for result in search.results():
        paper = {
            "title": result.title,
            "authors": ", ".join(author.name for author in result.authors),
            "published": result.published,
            "url": result.entry_id,
            "abstract": result.summary
        }
        papers.append(paper)
        abstracts.append(result.summary)
except Exception as e:
    print(f"エラーが発生しました: {e}")

# TF-IDFベクトル化
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(abstracts)

# クラスタ数の最適化
best_n_clusters = 2
best_score = -1
for n_clusters in range(2, min(10, len(abstracts))):  # 最大10クラスタに限定
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    if score > best_score:
        best_score = score
        best_n_clusters = n_clusters

# 最適なクラスタ数でKMeans実行
kmeans = KMeans(n_clusters=best_n_clusters, random_state=42)
labels = kmeans.fit_predict(X)

# 各クラスタに対応するドメイン名を決定するためのマッピング
cluster_to_domain = {
    0: "AI and ML",
    1: "NLP",
    2: "Computer Vision",
    3: "Ethics and Safety",
    4: "Applications",
    # 必要に応じて拡張
}

# Markdown形式のテーブルを作成
markdown_table = "| 論文タイトル | 著者 | 発行日 | URL | ドメイン |\n"
markdown_table += "|-------------|------|--------|-----|----------|\n"

# 結果の表示とテーブルの作成
for i, paper in enumerate(papers):
    domain = cluster_to_domain.get(labels[i], "未分類")
    print(f"Title: {paper['title'][:30]}..., Domain: {domain}")
    markdown_table += f"| {paper['title']} | {paper['authors']} | {paper['published'].date()} | [Link]({paper['url']}) | {domain} |\n"

# 最終的なテーブルを表示
print("\n最終的なMarkdownテーブル:\n")
print(markdown_table)

In [ ]:
### PubMedから該当する論文のみを抜き出し

#!pip install requests autogen --q

import requests
import xml.etree.ElementTree as ET
from google.colab import files
import io
import csv
import autogen

# Autogenの設定
config_list = [
    {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"]
    }
]

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "config_list": config_list,
    }
)

human = autogen.UserProxyAgent(
    name="human",
    system_message="You are a helpful human assistant.",
    human_input_mode="NEVER"
)

def fetch_pubmed_articles(query, max_results=20):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "xml"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        id_list = [id_elem.text for id_elem in root.findall(".//Id")]
        return id_list
    else:
        print(f"PubMedからデータを取得する際にエラーが発生しました: {response.status_code}")
        return []

def fetch_article_details(article_ids):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(article_ids),
        "retmode": "xml"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        root = ET.fromstring(response.content)
        articles = []
        for article_elem in root.findall(".//PubmedArticle"):
            title_elem = article_elem.find(".//ArticleTitle")
            title = title_elem.text if title_elem is not None else "No Title"

            abstract_elem = article_elem.find(".//AbstractText")
            abstract = abstract_elem.text if abstract_elem is not None else "No Abstract"

            articles.append({"title": title, "abstract": abstract})

        return articles
    else:
        print(f"論文の詳細を取得する際にエラーが発生しました: {response.status_code}")
        return []

def check_relevance(title, abstract):
    prompt = f"""
    Title: {title}
    Abstract: {abstract}

    Based on the title and abstract above, determine if this article is specifically about the exchange or replacement of multifocal intraocular lenses (IOLs).
    The article should focus on the process, outcomes, or reasons for exchanging or replacing multifocal IOLs that have already been implanted.

    Respond with either 'Relevant' or 'Not Relevant', followed by a brief explanation of your decision.
    """

    human.initiate_chat(assistant, message=prompt, max_turns=1)
    response = assistant.last_message()["content"]

    is_relevant = response.lower().startswith("relevant")
    explanation = response.split("\n", 1)[1] if "\n" in response else ""

    return is_relevant, explanation

def save_to_csv(articles):
    output = io.StringIO()
    writer = csv.DictWriter(output, fieldnames=["title", "abstract", "relevant", "explanation"])
    writer.writeheader()
    for article in articles:
        writer.writerow(article)

    return output.getvalue()

# Example usage
query = '"multifocal IOL exchange"'
article_ids = fetch_pubmed_articles(query)
articles = fetch_article_details(article_ids)

# Check relevance of each article
for article in articles:
    is_relevant, explanation = check_relevance(article["title"], article["abstract"])
    article["relevant"] = "Yes" if is_relevant else "No"
    article["explanation"] = explanation
    print(f"Title: {article['title']}")
    print(f"Relevant: {article['relevant']}")
    print(f"Explanation: {article['explanation']}")
    print()

# Save to CSV and download
csv_content = save_to_csv(articles)
with open('articles.csv', 'w', newline='', encoding='utf-8') as f:
    f.write(csv_content)

files.download('articles.csv')

human (to assistant):


    Title: Refractive outcomes after multifocal intraocular lens exchange.
    Abstract: To evaluate the refractive outcomes after multifocal intraocular lens (IOL) exchange.

    Based on the title and abstract above, determine if this article is specifically about the exchange or replacement of multifocal intraocular lenses (IOLs). 
    The article should focus on the process, outcomes, or reasons for exchanging or replacing multifocal IOLs that have already been implanted.
    
    Respond with either 'Relevant' or 'Not Relevant', followed by a brief explanation of your decision.
    

--------------------------------------------------------------------------------
assistant (to human):

Relevant. The title and abstract specifically indicate that the article evaluates the refractive outcomes after the exchange of multifocal intraocular lenses (IOLs). This suggests that the focus is on the results of the procedure of exchanging or replacing multifocal IOLs tha

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Webを調べるコード

In [22]:
# Import necessary libraries
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

llm_config = {
    "request_timeout": 600,
    "seed": 42,
    "config_list":  [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}],
    "temperature": 0,
}

# Construct agents
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

# Start a conversation
user_proxy.initiate_chat(
    assistant,
    message="""
Tell me about this project, and the libary, then also tell me what I can use it for: https://www.amed.go.jp/about-navi.html
""",
)

user_proxy (to assistant):


Tell me about this project, and the libary, then also tell me what I can use it for: https://www.amed.go.jp/about-navi.html


--------------------------------------------------------------------------------
assistant (to user_proxy):

To provide you with detailed information about the project and the library from the provided URL, I will access the webpage and extract relevant content. Let's investigate the webpage content now.

```python
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://www.amed.go.jp/about-navi.html"

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Print out the page title and first few paragraphs to understand the content
    print("Page Title:", soup.title.string)
    paragraphs = soup.find_all('p')
    for i, p in enu

user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Page Title: AMEDを知りたい方 | 国立研究開発法人日本医療研究開発機構
Paragraph 1: 
Paragraph 2: 分野別紹介
Paragraph 3: 事業部紹介
Paragraph 4: 公募情報検索
Paragraph 5: 公募情報一覧


--------------------------------------------------------------------------------
assistant (to user_proxy):

The webpage from the URL you provided is titled "AMEDを知りたい方 | 国立研究開発法人日本医療研究開発機構", which translates to "For those who want to know about AMED | Japan Agency for Medical Research and Development". AMED stands for the Japan Agency for Medical Research and Development.

From the titles of the sections:
1. **分野別紹介** - Introduction by Field
2. **事業部紹介** - Introduction of Business/Project Divisions
3. **公募情報検索** - Call for Proposals Information Search
4. **公募情報一覧** - List of Call for Proposals

#### Overview:
The website appears to be an informational resource provided by AMED, aimed at introducing various aspects of their work and initiatives, including different fields and

KeyboardInterrupt: Interrupted by user

In [28]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, GroupChat, GroupChatManager
import os

gpt4o_config = {
    "seed": 42,
    "temperature": 0,
    "config_list":  [{"model": "gpt-4o", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}],
    "request_timeout": 120,
}

user_proxy = UserProxyAgent(
   name="Admin",
   system_message="人間の管理者。プランナーと対話してプランを議論します。プランの実行には管理者の承認が必要です。",
   code_execution_config=False,
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4o_config,
    system_message='''エンジニア。承認されたプランに従います。タスクを解決するためにPython/シェルコードを書きます。コードはスクリプトタイプを指定するコードブロックで囲みます。ユーザーはあなたのコードを修正できません。そのため、他の人が修正する必要がある不完全なコードを提案しないでください。実行者が実行することを意図していない場合は、コードブロックを使用しないでください。
1つの応答に複数のコードブロックを含めないでください。他の人に結果をコピーして貼り付けるように頼まないでください。実行者が返した実行結果を確認してください。
結果にエラーがあることが示されている場合は、エラーを修正してコードを再度出力します。部分的なコードやコードの変更ではなく、完全なコードを提案してください。エラーが修正できない場合、またはコードが正常に実行された後でもタスクが解決されない場合は、問題を分析し、仮定を見直し、必要な追加情報を収集し、試すべき別のアプローチを考えてください。
日本語で話します。
''',
)

scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4o_config,
    system_message="""科学者。承認されたプランに従います。抄録を見た後、論文を分類することができます。コードは書きません。日本語で話します。"""
)

planner = AssistantAgent(
    name="Planner",
    system_message='''プランナー。プランを提案します。管理者と批評家からのフィードバックに基づいてプランを修正し、管理者の承認を得るまで続けます。
プランには、コードを書くことができるエンジニアとコードを書かない科学者が関与する場合があります。
まずプランを説明してください。クエリデザイナー/リサーチャーが実行するステップとエンジニアが実行するステップと科学者が実行するステップを明確にしてください。
実際に問い合わせをしたりインタビューしたりはできません。web内のデータ収集で完結できる範囲内で計画を立ててください。
プランは日本語で記述してください。
''',
    llm_config=gpt4o_config,
)

executor = UserProxyAgent(
    name="Executor",
    system_message="実行者。エンジニアが書いたコードを実行し、結果を報告します。",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)

critic = AssistantAgent(
    name="Critic",
    system_message="批評家。他のエージェントからのプラン、主張、コードをダブルチェックし、元の目的から外れていないかどうかフィードバックを提供します。プランにソースURLなどの検証可能な情報が含まれているかどうかを確認します。",
    llm_config=gpt4o_config,
)

# 新しいエージェントを追加
researcher = AssistantAgent(
    name="Researcher",
    llm_config=gpt4o_config,
    system_message="""研究者。query_designerからの提案をもとに、Webからの情報収集を担当します。検索しているサイトをリアルタイムに表示してください。提供された検索クエリを使用して情報を収集し、関連する情報を要約して報告します。情報源のURLも必ず含めてください。日本語で話します。"""
)

query_designer = AssistantAgent(
    name="QueryDesigner",
    llm_config=gpt4o_config,
    system_message="""クエリデザイナー。効果的な検索クエリを設計することを担当します。与えられたトピックや質問に基づいて、最も関連性の高い情報を得るための検索クエリを提案します。必要に応じて複数のクエリを提案することもあります。日本語で話します。"""
)

groupchat = GroupChat(agents=[user_proxy, engineer, scientist, planner, executor, critic, researcher, query_designer], messages=[], max_round=50)
manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4o_config)

user_proxy.initiate_chat(
    manager,
    message="""
吹田市の寺の数について考察
""",
)

Admin (to chat_manager):


吹田市の寺の数について考察


--------------------------------------------------------------------------------
Planner (to chat_manager):

プロジェクトの目的は、吹田市にある寺の数を把握し、それらの位置や特徴を考察することです。以下に、プロジェクトのステップを示します。

### ステップ1: データ収集 (リサーチャー)
1. **Web検索による情報収集**
   - 吹田市の公式ウェブサイトや観光情報サイト、Wikipediaなどから寺の数に関する基本情報を集めます。
   - 日本全国の寺院データを取り扱っているデータベース（例: 寺院データベース）を利用して、吹田市内の寺院リストを取得します。

2. **オープンデータの活用**
   - 吹田市や大阪府が提供しているオープンデータポータルを確認し、寺院に関するデータセットがあるか探します。

### ステップ2: データの整理と分析 (エンジニア)
1. **データクレンジング**
   - 収集したデータを整理し、重複や不正確な情報を排除します。
   - 必要に応じてデータをフォーマット化し、統一された形式に整えます。

2. **地理情報の抽出とマッピング**
   - 各寺の緯度と経度を取得し、地図上にプロットするための準備を行います。
   - GISソフトウェアやGoogle Maps APIを用いて、寺の位置を可視化します。

### ステップ3: 分析結果の考察 (科学者)
1. **特徴分析**
   - 吹田市の寺院の特徴や傾向を分析します。例えば、寺の密集度、歴史的重要性、観光地としての人気度などを考察します。

2. **レポート作成**
   - 分析結果をまとめ、レポートを作成します。レポートには、データの可視化（地図やグラフ）も含めます。

### ステップ4: 結果のフィードバックと公開 (管理者)
1. **レビューと修正**
   - 作成されたレポートをレビューし、必要に応じて修正を指示します。

2. **公開と共有**
   - レポートをウェブサイトやソーシャルメディアで公開し、一般市民や興味を持つ

Executor (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
吹田市の寺院数: 4


--------------------------------------------------------------------------------
Scientist (to chat_manager):

結果を確認しました。仮のデータに基づいて、重複を排除した結果、吹田市の寺院数が「4」となりました。これは仮のデータを使用していますので、実際のデータが手に入った場合、そのデータに基づいて再度確認することが重要です。

今後のステップとしては、実際のデータを収集した後、このコードを適用して正確な寺院数を特定することが求められます。データが整い次第、再度確認を行い、確定した寺院数を報告します。

--------------------------------------------------------------------------------
Critic (to chat_manager):

このプランと実行結果について確認しましたが、仮のデータを使用してのテスト結果であるため、実際の吹田市の寺院数を特定するためには、もっと具体的なデータが必要です。現時点で具体的な寺院数を示すソースURLやデータセットに関する情報がないため、次のアクションとしては、現実の寺院データを収集し、それに基づいて再度評価を行うことが必要です。また、正式な情報源を確認することも重要です。各ステップに関連するデータソースの具体的なリンクや証拠を示すことが必要です。次回の実行時には、実際のデータを用いて結果を出すことを目指しましょう。

--------------------------------------------------------------------------------
QueryDesigner (to chat_manager):

フィードバックありがとうございます。実際の吹田市の寺院数を特定するため、具体的なデータ収集に焦点を当てたプランを再度考慮します。

### 次のステップ:
1. **公式情報源の特定**:
   - 吹田市の公式サイトや観光協会の

KeyboardInterrupt: Interrupted by user

In [13]:
engineer.reset()
scientist.reset()
planner.reset()
executor.reset()
critic.reset()
researcher.reset()
query_designer.reset()